<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Chapter 9: Business Alignment

**Purpose:** Align data exploration with business objectives and constraints.

**Outputs:**
- Business context documentation
- Success metrics definition
- Constraints and requirements

---

## 9.1 Setup

In [1]:
from customer_retention.analysis.auto_explorer import ExplorationFindings
from customer_retention.analysis.visualization import display_table
import pandas as pd

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
# === CONFIGURATION ===
from pathlib import Path

FINDINGS_DIR = Path("../experiments/findings")

findings_files = [f for f in FINDINGS_DIR.glob("*_findings.yaml") if "multi_dataset" not in f.name]
if not findings_files:
    raise FileNotFoundError(f"No findings files found in {FINDINGS_DIR}. Run notebook 01 first.")

findings_files.sort(key=lambda f: f.stat().st_mtime, reverse=True)
FINDINGS_PATH = str(findings_files[0])

print(f"Found {len(findings_files)} findings file(s)")
print(f"Using: {FINDINGS_PATH}")

findings = ExplorationFindings.load(FINDINGS_PATH)

print(f"\nLoaded findings for {findings.column_count} columns")

FileNotFoundError: No findings files found in ../experiments/findings. Run notebook 01 first.

## 9.2 Business Context

Define the business context for this project.

In [ ]:
BUSINESS_CONTEXT = {
    "project_name": "Customer Churn Prediction",
    "business_objective": "Reduce customer churn by 20% through proactive retention campaigns",
    "stakeholders": ["Marketing Team", "Customer Success", "Data Science"],
    "timeline": "Q1 2025",
    "budget_constraints": "$50k for retention campaigns per month"
}

print("Business Context:")
for key, value in BUSINESS_CONTEXT.items():
    print(f"  {key}: {value}")

## 9.3 Success Metrics

In [ ]:
SUCCESS_METRICS = [
    {
        "Metric": "Model AUC",
        "Target": ">= 0.80",
        "Priority": "High",
        "Rationale": "Need strong discrimination to prioritize high-risk customers"
    },
    {
        "Metric": "Precision at 20%",
        "Target": ">= 0.60",
        "Priority": "High",
        "Rationale": "Limited budget means we can only target top 20% of predictions"
    },
    {
        "Metric": "Churn Rate Reduction",
        "Target": "20%",
        "Priority": "High",
        "Rationale": "Primary business objective"
    },
    {
        "Metric": "Model Latency",
        "Target": "< 100ms",
        "Priority": "Medium",
        "Rationale": "Required for real-time scoring"
    },
    {
        "Metric": "Fairness (Demographic Parity)",
        "Target": "Ratio >= 0.8",
        "Priority": "Medium",
        "Rationale": "Ensure equitable treatment across segments"
    }
]

metrics_df = pd.DataFrame(SUCCESS_METRICS)
print("Success Metrics:")
display(metrics_df)

## 9.4 Deployment Requirements

In [ ]:
DEPLOYMENT_REQUIREMENTS = {
    "scoring_mode": "Both batch and real-time",
    "batch_frequency": "Daily",
    "real_time_latency": "< 100ms p99",
    "infrastructure": "Databricks",
    "model_registry": "MLflow",
    "monitoring": "Required - drift detection and performance tracking",
    "retraining": "Monthly or on significant drift"
}

print("Deployment Requirements:")
for key, value in DEPLOYMENT_REQUIREMENTS.items():
    print(f"  {key}: {value}")

## 9.5 Data Constraints

In [ ]:
DATA_CONSTRAINTS = [
    {
        "Constraint": "PII Handling",
        "Requirement": "No direct PII in features (names, SSN, etc.)",
        "Status": "To verify"
    },
    {
        "Constraint": "Data Freshness",
        "Requirement": "Features must be available within 24 hours",
        "Status": "To verify"
    },
    {
        "Constraint": "Historical Depth",
        "Requirement": "Minimum 12 months of history for training",
        "Status": "To verify"
    },
    {
        "Constraint": "Protected Attributes",
        "Requirement": "Age, gender, race should not be direct features",
        "Status": "To verify"
    }
]

constraints_df = pd.DataFrame(DATA_CONSTRAINTS)
print("Data Constraints:")
display(constraints_df)

## 9.6 Intervention Strategy

In [ ]:
INTERVENTIONS = [
    {
        "Risk Level": "High (>0.8)",
        "Intervention": "Personal call from account manager",
        "Cost": "$50/customer",
        "Expected Effectiveness": "40% retention"
    },
    {
        "Risk Level": "Medium (0.5-0.8)",
        "Intervention": "Personalized email + discount offer",
        "Cost": "$10/customer",
        "Expected Effectiveness": "20% retention"
    },
    {
        "Risk Level": "Low (<0.5)",
        "Intervention": "Automated engagement email",
        "Cost": "$0.50/customer",
        "Expected Effectiveness": "5% retention"
    }
]

interventions_df = pd.DataFrame(INTERVENTIONS)
print("Intervention Strategy:")
display(interventions_df)

## 9.7 Save Business Context to Findings

In [ ]:
findings.metadata = findings.metadata or {}
findings.metadata["business_context"] = BUSINESS_CONTEXT
findings.metadata["success_metrics"] = SUCCESS_METRICS
findings.metadata["deployment_requirements"] = DEPLOYMENT_REQUIREMENTS

findings.save(FINDINGS_PATH)
print(f"Business context saved to: {FINDINGS_PATH}")

---

## Next Steps

Continue to **10_spec_generation.ipynb** to generate production specifications.